##### TASK 1: Create Catalog & Schemas

- Trying to create a new catalog. If this fails due to permissions, we will be using our existing catalog (e.g., USE CATALOG workspace;)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ecommerce;
USE CATALOG ecommerce;

-- Create the 3 Schemas for our Medallion Architecture
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

-- Verify they exist
SHOW SCHEMAS;

databaseName
bronze
default
gold
information_schema
silver


##### TASK 2: Register Delta Tables
- We are not moving data. We are just pointing the Metastore to where our Delta files already sit (from Day 7).

In [0]:
%sql

-- 1. Register Bronze Table
DROP TABLE IF EXISTS bronze.raw_traffic;
CREATE TABLE bronze.raw_traffic 
AS SELECT * FROM delta.`/Volumes/workspace/ecommerce/ecommerce_data/medallion/bronze`;

-- 2. Register Silver Table
DROP TABLE IF EXISTS silver.cleaned_traffic;
CREATE TABLE silver.cleaned_traffic 
AS SELECT * FROM delta.`/Volumes/workspace/ecommerce/ecommerce_data/medallion/silver`;

-- 3. Register Gold Table
DROP TABLE IF EXISTS gold.product_kpis;
CREATE TABLE gold.product_kpis 
AS SELECT * FROM delta.`/Volumes/workspace/ecommerce/ecommerce_data/medallion/gold`;

-- Verify
SELECT * FROM gold.product_kpis LIMIT 5;

product_id,category_code,brand,total_views,total_carts,total_purchases,total_revenue,conversion_rate
1005159,null,null,35160,null,null,1.3512530220000004E7,384.32
8300340,null,null,15,null,null,3294.8,219.65
5701087,null,null,3443,null,null,252251.99999999927,73.27
28719030,null,null,79,null,null,6363.62,80.55
27300009,null,null,353,null,null,99710.02,282.46


##### TASK 3: Set Up Permissions (Governance) 
- Unity Catalog lets you control who sees what.
- Grant full access on Silver to Engineers (Example)
  - Note: If these groups don't exist in your workspace, this might warn/fail. 
- We can replace `engineers` with your own email for testing.

In [0]:
%sql
-- GRANT ALL PRIVILEGES ON SCHEMA silver TO `account users`;

-- Grant Read-Only access on Gold to Analysts
GRANT SELECT ON TABLE gold.product_kpis TO `account users`;

-- Check your own grants
SHOW GRANTS ON TABLE gold.product_kpis;

Principal,ActionType,ObjectType,ObjectKey
account users,SELECT,TABLE,ecommerce.gold.product_kpis


##### TASK 4: Create Views for Controlled Access
- Views let you hide sensitive columns (like PII) or simplify complex queries.


In [0]:
%sql
-- Create a view that only shows High Performing products
CREATE OR REPLACE VIEW gold.top_performing_products AS
SELECT 
    product_id,
    total_revenue,
    conversion_rate
FROM gold.product_kpis
WHERE conversion_rate > 0.05
ORDER BY total_revenue DESC;

-- Test the view
SELECT * FROM gold.top_performing_products LIMIT 10;

product_id,total_revenue,conversion_rate
1005115,3.51415041479992E8,1499.22
1005105,3.026340283900096E8,2043.72
1005135,1.877934551700019E8,2460.99
1004249,1.706257307599948E8,1179.4
1004767,1.0870326279999979E8,455.52
1003317,9.99628432799985E7,1354.27
1002544,9.440216534999782E7,774.44
1004659,7.503532500000001E7,1133.46
1002524,7.139219288999987E7,934.97
1003306,6.737110682000002E7,932.77
